# Grid Rival

In [1]:
import copy
import pandas as pd
import numpy as np

In [2]:
from itertools import combinations, product

In [17]:
from gridrival.probabilities import GridProbabilities
from gridrival.probabilities.betting_odds import WinningOdds, TOP_1
from gridrival.drivers import DRIVERS, PGasly, CSainz, YTsunoda, GRUssell, MVerstappen
from gridrival.fantasy import FantasyTeam
from gridrival.teams import TEAMS, Ferrari

In [4]:
Top1Odds = WinningOdds(pd.Series(TOP_1))

In [5]:
Prob = GridProbabilities(Top1Odds.naive_grid())

In [6]:
for driver in DRIVERS:
    driver.probabilities = Prob.driver_probabilities(driver.name)

In [7]:
fixed_drivers = [driver.to_fixed_info() for driver in DRIVERS]
fixed_teams = [team.to_fixed_info() for team in TEAMS]

In [26]:
universe = product(combinations(fixed_drivers, 5), fixed_teams)

In [27]:
ft_universe = [FantasyTeam(comb[0], comb[1]) for comb in universe]

In [30]:
budget_universe = [ft for ft in ft_universe if ft.cost() <= 103.4*1e6]

In [33]:
constraint_universe = []

for ft in budget_universe:

    if any(in_element not in ft for in_element in [PGasly, Ferrari]):
        continue
    if any(out_element in ft for out_element in [MVerstappen, CSainz, YTsunoda, GRUssell]):
        continue
    constraint_universe.append(ft)

In [38]:
restricted_universe = [team for team in ft_universe if PGasly in team and Ferrari in team and MVerstappen not in team and CSainz not in team and YTsunoda not in team and GRUssell not in team]

In [39]:
budget_universe = [team for team in restricted_universe if team.cost() <= 103.4*1e6]

In [40]:
len(constraint_universe)

692

In [41]:
len(budget_universe)

692

In [35]:
data = [
    {
        "driver_1": ft.drivers[0],
        "driver_2": ft.drivers[1],
        "driver_3": ft.drivers[2],
        "driver_4": ft.drivers[3],
        "driver_5": ft.drivers[4],
        "team": ft.team,
        "talent_driver": ft.talent_driver,
        "cost": ft.cost(),
        "points": ft.points(),
    }
    for ft in constraint_universe
]

In [36]:
df = pd.DataFrame(data)

In [37]:
df[df["cost"]<=103.4*1e6].sort_values("points", ascending=False)

,driver_1,driver_2,driver_3,driver_4,driver_5,team,talent_driver,cost,points
177,S. Perez,D. Ricciardo,P. Gasly,E. Ocon,N. Latifi,Ferrari,P. Gasly,102900000.0,911.362117
81,V. Bottas,D. Ricciardo,P. Gasly,E. Ocon,N. Mazepin,Ferrari,P. Gasly,101900000.0,910.925363
206,S. Perez,L. Norris,P. Gasly,E. Ocon,N. Latifi,Ferrari,P. Gasly,103000000.0,910.773338
176,S. Perez,D. Ricciardo,L. Stroll,P. Gasly,N. Mazepin,Ferrari,P. Gasly,103200000.0,910.339654
103,V. Bottas,L. Norris,P. Gasly,E. Ocon,N. Mazepin,Ferrari,P. Gasly,102000000.0,910.336584
...,...,...,...,...,...,...,...,...,...
688,P. Gasly,K. Raikkonen,A. Giovinazzi,N. Latifi,N. Mazepin,Ferrari,P. Gasly,67500000.0,792.431719
687,P. Gasly,K. Raikkonen,A. Giovinazzi,N. Latifi,M. Schumacher,Ferrari,P. Gasly,70100000.0,785.907204
689,P. Gasly,K. Raikkonen,A. Giovinazzi,M. Schumacher,N. Mazepin,Ferrari,P. Gasly,67700000.0,782.657330
691,P. Gasly,A. Giovinazzi,N. Latifi,M. Schumacher,N. Mazepin,Ferrari,P. Gasly,60500000.0,777.151301
